This notebook first appends the keystroke to the database and fetches the keystroke of the user based on the user_name 
from the database and runs the siamese model on it which is also loaded. 

To-do: Add the UI and data collection script before this is done and test the en

In [104]:
import json
import csv
import os
import pandas as pd
import numpy as np

In [105]:
def read_data():
    user_name = 'mahathi'
#     subject_name = input("Enter username whose data is to be appended! :")
#     subject_files = []
#     for filename in os.listdir("./output/"):
#         if filename.startswith(subject_name):
#             subject_files.append(filename)

    # print(rows)
    csv_dd_rows = []
    csv_ud_rows = []
    csv_ht_rows = []
    all_keys = []
    # csv_row = []
    with open('DSL-StrongPasswordData.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            for item in row:
                all_keys.append(item)
                if item.startswith("H."):
                    csv_ht_rows.append(item)
                elif item.startswith("DD."):
                    csv_dd_rows.append(item)
                elif item.startswith("UD."):
                    csv_ud_rows.append(item)

            break
    csv_file.close()

    # print(csv_ud_rows)
    # print(csv_dd_rows)
    # print(csv_ht_rows)

    rows = []

    hold_time_user = []
    dd_key1_key2_user = []
    ud_key1_key2_user = []

    cnt = 1
    try:
        with open('output/{}_timings.json'.format(user_name), 'r') as readfile:
            hold_time_user_dict = dd_key1_key2_dict = ud_key1_key2_dict = {}
            data = json.load(readfile)
            for timing in data['timings']:
                hold_time_user.append(timing["hold_time"])
                dd_key1_key2_user.append(timing["dd_key1_key2"])
                ud_key1_key2_user.append(timing["ud_key1_key2"])
                row = dict()
                for key in timing["hold_time"].keys():
                    if key == "5":
                        new_key = "H.five"
                    elif key == "R":
                        new_key = "H.Shift.r"
                    else:
                        new_key = "H."+key
                    row[new_key] = timing["hold_time"][key]
                for key in timing["dd_key1_key2"].keys():
                    if key == "DD.5.R":
                        new_key = "DD.five.Shift.r"
                    elif key == "DD.R.o":
                        new_key = "DD.Shift.r.o"
                    elif key == "DD.e.5":
                        new_key = "DD.e.five"
                    else:
                        new_key = key
                    row[new_key] = timing["dd_key1_key2"][key]
                for key in timing["ud_key1_key2"].keys():
                    if key == "UD.5.R":
                        new_key = "UD.five.Shift.r"
                    elif key == "UD.R.o":
                        new_key = "UD.Shift.r.o"
                    elif key == "UD.e.5":
                        new_key = "UD.e.five"
                    else:
                        new_key = key
                    row[new_key] = timing["ud_key1_key2"][key]
                    # print(key)
                row["subject"] = user_name
                row["rep"] = cnt
                row["sessionIndex"] = 1
                rows.append(row)
                cnt += 1
    except FileNotFoundError as fnfe:
        print("FileNotFoundError")


    # csv_file = "DSL-StrongPasswordData_mod.csv"
    # try:
    #     with open(csv_file, 'w') as csvfile:
    #         writer = csv.DictWriter(csvfile, fieldnames=all_keys)
    #         writer.writeheader()
    #         for data in rows:
    #             writer.writerow(data)
    # except IOError:
    #     print("I/O error")


    edit_csv_file = "edited_dataset/DSL-StrongPasswordData.csv"
    try:
        with open(edit_csv_file, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=all_keys)
            # writer.writeheader()
            for data in rows:
                writer.writerow(data)
    except IOError:
        print("I/O error")

    csvfile.close()
    data = pd.read_csv(edit_csv_file)
    return data

In [106]:
data = read_data()

In [107]:
data.tail()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
20455,mahathi,1,6,0.088738,1.226724,1.315462,0.105833,0.683766,0.789599,0.099846,...,0.132816,0.117374,0.250190,0.103507,0.990037,1.093544,0.084784,0.187926,0.272710,0.079048
20456,mahathi,1,7,0.082551,0.721624,0.804175,0.117554,0.300890,0.418444,0.126766,...,0.141494,0.211331,0.352825,0.132153,0.898021,1.030174,0.095169,0.204375,0.299544,0.089683
20457,mahathi,1,8,0.098981,0.381759,0.480740,0.115835,0.241615,0.357450,0.109227,...,0.132030,0.116725,0.248755,0.121086,0.883652,1.004739,0.088824,0.182029,0.270853,0.089959
20458,mahathi,1,9,0.092467,0.866233,0.958700,0.122159,0.260265,0.382425,0.115951,...,0.126474,0.160943,0.287418,0.101626,0.330910,0.432536,0.137388,0.273862,0.411250,0.081497
20459,mahathi,1,10,0.102621,1.302548,1.405169,0.107285,0.213225,0.320510,0.097653,...,0.150363,0.152965,0.303328,0.113600,0.676194,0.789794,0.121900,0.609356,0.731256,0.093484


In [108]:
# x2 and x3 are mahathi's keystroke x1 and x2 are imposter

x1 = np.array(data.loc[data.subject == 's054', 'H.period':'H.Return'][1:2]) #subj is user_name
x2 = np.array(data.iloc[-3:-2,3:])
x4 = np.array(data.iloc[-5:-4,3:])
x3 = np.array(data.loc[data.subject == 's054', 'H.period':'H.Return'][3:4])

In [109]:
x2.shape

(1, 31)

In [110]:
from tensorflow.keras.models import model_from_json

In [111]:
json_file = open('model_architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_weights.h5")
print("Loaded model from disk")
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


In [112]:
# evaluate loaded model on test data
score = loaded_model.predict([x1, x2])
np.rint(score)

array([[0.]], dtype=float32)